In [1]:
from random_stroke_generator import RandomStrokeGenerator

training_gen = RandomStrokeGenerator(batch_size=484,
                                     num_data=12100, 
                                     min_strokes=32, 
                                     max_strokes=128, 
                                     jumping_rate=0.1,
                                     max_jumping_step=41
                                    )
valid_gen = RandomStrokeGenerator(batch_size=484,
                                  num_data=4840, 
                                  min_strokes=32, 
                                  max_strokes=128,
                                  jumping_rate=0.1,
                                  max_jumping_step=41
                                )

In [2]:
global_size = (84,84,4)
local_size = (11,11,2)

# Just latest RSG2

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Concatenate, Input, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
import matplotlib.pyplot as plt
import numpy as np

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

class DoodleSDQModel():
    def __init__(self, name, global_input_size, local_input_size, output_size):
        #super(DoodleSDQModel, self).__init__()
        self.name = name
        self.global_input_size = global_input_size
        self.local_input_size = local_input_size
        self.output_size = output_size
        
        self.model = self._create_model()
        
        # class weight: down=1, up=0.5
#         self.class_weight = {i:1 if i<121 else 0.5 for i in range(output_size)}
        
        
    def _create_model(self):
        local_input = tf.keras.Input(shape=self.local_input_size)
        global_input = tf.keras.Input(shape=self.global_input_size)
        
        inputs = [global_input, local_input]
        
        x_local = self._create_local_CNN(local_input)
        x_global = self._create_global_CNN(global_input)
        
        x = Concatenate(axis=1)([x_global, x_local])
        
        x = Dropout(.2)(x)
        x = Dense(2048, activation='relu')(x)
        x = Dense(self.output_size)(x)
        outputs = Activation("softmax") (x)
        return Model(inputs, outputs, name="doodle_dqn")

    def _create_global_CNN(self, input):
        
        x = Conv2D(32, (8,8), strides=(4,4), padding='same', activation='relu')(input)
        # x = Activation("relu")(x)
        # x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (4,4), strides=(2,2), padding='same', activation='relu')(x)
        # x = Activation("relu")(x)
        # x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu')(x)
        # x = Activation("relu")(x)
        # x = MaxPooling2D((2,2))(x)
        
        x = Flatten()(x)
        
        return x

    def _create_local_CNN(self, input):
        x = Conv2D(256, (11,11), strides=(1,1), padding='same', activation='relu')(input)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(256, (3,3), strides=(1,1), padding='same', activation='relu')(x)
        # x = MaxPooling2D((2,2))(x)
        # x = Activation("relu")(x)
        # x = GlobalAveragePooling2D()(x)
        
        x = Flatten()(x)
        return x
    
    def train(self,training_gen, validation_data, epochs, init_epoch=0):
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss',min_delta=0, patience=8, verbose=1,)
        tensorboard = TensorBoard(
                              log_dir='logs/'+self.name,
                              histogram_freq=0,
                              write_images=True
                            )
        
        self.epochs = epochs
        self.model.compile(optimizer=Adam(lr=.003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.history = self.model.fit(training_gen,\
                                      validation_data=validation_data, \
                                      validation_steps= len(validation_data), \
                                      epochs=epochs, \
                                      initial_epoch=init_epoch,\
                                      callbacks=[reduce_lr, tensorboard], \
#                                       class_weight=self.class_weight, \
                                      verbose=1)
    
    def plot(self):
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']

        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        epochs_range = range(self.epochs)

        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right')
        plt.title('Training and Validation Accuracy')

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.legend(loc='upper right')
        plt.title('Training and Validation Loss')
        plt.show()


In [4]:
name = "latest_RSG2"
model_path = 'experiments/supervised/' + name + ".h5"
model = DoodleSDQModel(name, global_input_size=global_size, local_input_size=local_size, output_size=242)
# model.model.summary()
model.train(training_gen, valid_gen, 50, init_epoch=-1)
model.model.save(model_path)

Epoch 0/50
25/25 [==============================] - 4s 141ms/step - loss: 4.7078 - accuracy: 0.0487 - val_loss: 3.8780 - val_accuracy: 0.0897 - lr: 0.0030
Epoch 1/50
25/25 [==============================] - 3s 126ms/step - loss: 3.6692 - accuracy: 0.1129 - val_loss: 3.3180 - val_accuracy: 0.1589 - lr: 0.0030
Epoch 2/50
25/25 [==============================] - 3s 124ms/step - loss: 2.9915 - accuracy: 0.1888 - val_loss: 2.6411 - val_accuracy: 0.2364 - lr: 0.0030
Epoch 3/50
25/25 [==============================] - 3s 123ms/step - loss: 2.5174 - accuracy: 0.2453 - val_loss: 2.3721 - val_accuracy: 0.2756 - lr: 0.0030
Epoch 4/50
25/25 [==============================] - 3s 123ms/step - loss: 2.3130 - accuracy: 0.2823 - val_loss: 2.2662 - val_accuracy: 0.2868 - lr: 0.0030
Epoch 5/50
25/25 [==============================] - 3s 127ms/step - loss: 2.1909 - accuracy: 0.3073 - val_loss: 2.1029 - val_accuracy: 0.3236 - lr: 0.0030
Epoch 6/50
25/25 [==============================] - 3s 129ms/step - lo

# MaxPooling Global and Global 

In [5]:

class DoodleSDQModel():
    def __init__(self, name, global_input_size, local_input_size, output_size):
        #super(DoodleSDQModel, self).__init__()
        self.name = name
        self.global_input_size = global_input_size
        self.local_input_size = local_input_size
        self.output_size = output_size
        
        self.model = self._create_model()
        
        # class weight: down=1, up=0.5
#         self.class_weight = {i:1 if i<121 else 0.5 for i in range(output_size)}
        
        
    def _create_model(self):
        tf.keras.backend.clear_session()
        local_input = tf.keras.Input(shape=self.local_input_size)
        global_input = tf.keras.Input(shape=self.global_input_size)
        
        inputs = [global_input, local_input]
        
        x_local = self._create_local_CNN(local_input)
        x_global = self._create_global_CNN(global_input)
        
        x = Concatenate(axis=1)([x_global, x_local])
        
        x = Dropout(.2)(x)
        x = Dense(2048, activation='relu')(x)
        x = Dense(self.output_size)(x)
        outputs = Activation("softmax") (x)
        return Model(inputs, outputs, name="doodle_dqn")

    def _create_global_CNN(self, input):
        
        x = Conv2D(32, (8,8), strides=(4,4), padding='same', activation='relu')(input)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (4,4), strides=(2,2), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)
        
        x = Flatten()(x)
        
        return x

    def _create_local_CNN(self, input):
        x = Conv2D(256, (11,11), strides=(1,1), padding='same', activation='relu')(input)
        x = GlobalAveragePooling2D()(x)
        
        x = Flatten()(x)
        return x
    
    def train(self,training_gen, validation_data, epochs, init_epoch=0):
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss',min_delta=0, patience=8, verbose=1,)
        tensorboard = TensorBoard(
                              log_dir='logs/'+self.name,
                              histogram_freq=0,
                              write_images=True
                            )
        
        self.epochs = epochs
        self.model.compile(optimizer=Adam(lr=.003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.history = self.model.fit(training_gen,\
                                      validation_data=validation_data, \
                                      validation_steps= len(validation_data), \
                                      epochs=epochs, \
                                      initial_epoch=init_epoch,\
                                      callbacks=[reduce_lr, tensorboard], \
#                                       class_weight=self.class_weight, \
                                      verbose=1)
    
    def plot(self):
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']

        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        epochs_range = range(self.epochs)

        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Training Accuracy')
        plt.plot(epochs_range, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right')
        plt.title('Training and Validation Accuracy')

        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Training Loss')
        plt.plot(epochs_range, val_loss, label='Validation Loss')
        plt.legend(loc='upper right')
        plt.title('Training and Validation Loss')
        plt.show()


In [6]:
name = "maxpooling_globalavgpooling"
model_path = 'experiments/supervised/' + name + ".h5"
model = DoodleSDQModel(name, global_input_size=global_size, local_input_size=local_size, output_size=242)
# model.model.summary()
model.train(training_gen, valid_gen, 50, init_epoch=-1)
model.model.save(model_path)

Epoch 0/50
25/25 [==============================] - 3s 132ms/step - loss: 4.2322 - accuracy: 0.0637 - val_loss: 3.8326 - val_accuracy: 0.0938 - lr: 0.0030
Epoch 1/50
25/25 [==============================] - 3s 116ms/step - loss: 3.7635 - accuracy: 0.0989 - val_loss: 3.5671 - val_accuracy: 0.1324 - lr: 0.0030
Epoch 2/50
25/25 [==============================] - 3s 117ms/step - loss: 3.4215 - accuracy: 0.1448 - val_loss: 3.1856 - val_accuracy: 0.1605 - lr: 0.0030
Epoch 3/50
25/25 [==============================] - 3s 118ms/step - loss: 3.0800 - accuracy: 0.1679 - val_loss: 3.0252 - val_accuracy: 0.1655 - lr: 0.0030
Epoch 4/50
25/25 [==============================] - 3s 118ms/step - loss: 2.9380 - accuracy: 0.1741 - val_loss: 2.7870 - val_accuracy: 0.1990 - lr: 0.0030
Epoch 5/50
25/25 [==============================] - 3s 118ms/step - loss: 2.8549 - accuracy: 0.1931 - val_loss: 2.7456 - val_accuracy: 0.1942 - lr: 0.0030
Epoch 6/50
25/25 [==============================] - 3s 116ms/step - lo

# AvgPooling and Global AvgPooling

In [7]:
from tensorflow.keras.layers import AveragePooling2D

class DoodleSDQModel():
    def __init__(self, name, global_input_size, local_input_size, output_size):
        #super(DoodleSDQModel, self).__init__()
        self.name = name
        self.global_input_size = global_input_size
        self.local_input_size = local_input_size
        self.output_size = output_size
        
        self.model = self._create_model()
        
        # class weight: down=1, up=0.5
#         self.class_weight = {i:1 if i<121 else 0.5 for i in range(output_size)}
        
        
    def _create_model(self):
        tf.keras.backend.clear_session()
        local_input = tf.keras.Input(shape=self.local_input_size)
        global_input = tf.keras.Input(shape=self.global_input_size)
        
        inputs = [global_input, local_input]
        
        x_local = self._create_local_CNN(local_input)
        x_global = self._create_global_CNN(global_input)
        
        x = Concatenate(axis=1)([x_global, x_local])
        
        x = Dropout(.2)(x)
        x = Dense(2048, activation='relu')(x)
        x = Dense(self.output_size)(x)
        outputs = Activation("softmax") (x)
        return Model(inputs, outputs, name="doodle_dqn")

    def _create_global_CNN(self, input):
        
        x = Conv2D(32, (8,8), strides=(4,4), padding='same', activation='relu')(input)
        x = AveragePooling2D((2,2))(x)
        
        x = Conv2D(64, (4,4), strides=(2,2), padding='same', activation='relu')(x)
        x = AveragePooling2D((2,2))(x)
        
        x = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu')(x)
        x = AveragePooling2D((2,2))(x)
        
        x = Flatten()(x)
        
        return x

    def _create_local_CNN(self, input):
        x = Conv2D(256, (11,11), strides=(1,1), padding='same', activation='relu')(input)
        x = GlobalAveragePooling2D()(x)
        
        x = Flatten()(x)
        return x
    
    def train(self,training_gen, validation_data, epochs, init_epoch=0):
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss',min_delta=0, patience=8, verbose=1,)
        tensorboard = TensorBoard(
                              log_dir='logs/'+self.name,
                              histogram_freq=0,
                              write_images=True
                            )
        
        self.epochs = epochs
        self.model.compile(optimizer=Adam(lr=.003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.history = self.model.fit(training_gen,\
                                      validation_data=validation_data, \
                                      validation_steps= len(validation_data), \
                                      epochs=epochs, \
                                      initial_epoch=init_epoch,\
                                      callbacks=[reduce_lr, tensorboard], \
#                                       class_weight=self.class_weight, \
                                      verbose=1)


In [8]:
name = "avgpooling_globalavgpooling"
model_path = 'experiments/supervised/' + name + ".h5"
model = DoodleSDQModel(name, global_input_size=global_size, local_input_size=local_size, output_size=242)
# model.model.summary()
model.train(training_gen, valid_gen, 50, init_epoch=-1)
model.model.save(model_path)

Epoch 0/50
25/25 [==============================] - 3s 129ms/step - loss: 4.2864 - accuracy: 0.0536 - val_loss: 3.8942 - val_accuracy: 0.0948 - lr: 0.0030
Epoch 1/50
25/25 [==============================] - 3s 117ms/step - loss: 3.8040 - accuracy: 0.1031 - val_loss: 3.6760 - val_accuracy: 0.1271 - lr: 0.0030
Epoch 2/50
25/25 [==============================] - 3s 114ms/step - loss: 3.4941 - accuracy: 0.1416 - val_loss: 3.2337 - val_accuracy: 0.1579 - lr: 0.0030
Epoch 3/50
25/25 [==============================] - 3s 117ms/step - loss: 3.1037 - accuracy: 0.1636 - val_loss: 2.9214 - val_accuracy: 0.1872 - lr: 0.0030
Epoch 4/50
25/25 [==============================] - 3s 119ms/step - loss: 2.9270 - accuracy: 0.1793 - val_loss: 2.8237 - val_accuracy: 0.1998 - lr: 0.0030
Epoch 5/50
25/25 [==============================] - 3s 119ms/step - loss: 2.8392 - accuracy: 0.1934 - val_loss: 2.7610 - val_accuracy: 0.2056 - lr: 0.0030
Epoch 6/50
25/25 [==============================] - 3s 118ms/step - lo

# Same Number of Local and Global Features

In [9]:
class DoodleSDQModel():
    def __init__(self, name, global_input_size, local_input_size, output_size):
        #super(DoodleSDQModel, self).__init__()
        self.name = name
        self.global_input_size = global_input_size
        self.local_input_size = local_input_size
        self.output_size = output_size
        
        self.model = self._create_model()
        
        # class weight: down=1, up=0.5
#         self.class_weight = {i:1 if i<121 else 0.5 for i in range(output_size)}
        
        
    def _create_model(self):
        tf.keras.backend.clear_session()
        local_input = tf.keras.Input(shape=self.local_input_size)
        global_input = tf.keras.Input(shape=self.global_input_size)
        
        inputs = [global_input, local_input]
        
        x_local = self._create_local_CNN(local_input)
        x_global = self._create_global_CNN(global_input)
        
        x = Concatenate(axis=1)([x_global, x_local])
        
        x = Dropout(.2)(x)
        x = Dense(2048, activation='relu')(x)
        x = Dense(self.output_size)(x)
        outputs = Activation("softmax") (x)
        return Model(inputs, outputs, name="doodle_dqn")

    def _create_global_CNN(self, input):
        
        x = Conv2D(32, (8,8), strides=(4,4), padding='same', activation='relu')(input)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (4,4), strides=(2,2), padding='same', activation='relu')(x)
        x = MaxPooling2D((2,2))(x)
        
        x = Conv2D(64, (3,3), strides=(1,1), padding='same', activation='relu')(x)
        
        x = Flatten()(x)
        
        return x

    def _create_local_CNN(self, input):
        x = Conv2D(256, (11,11), strides=(1,1), padding='same', activation='relu')(input)
        x = GlobalAveragePooling2D()(x)
        
        x = Flatten()(x)
        return x
    
    def train(self,training_gen, validation_data, epochs, init_epoch=0):
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)
        early_stopping = EarlyStopping(monitor='val_loss',min_delta=0, patience=8, verbose=1,)
        tensorboard = TensorBoard(
                              log_dir='logs/'+self.name,
                              histogram_freq=0,
                              write_images=True
                            )
        
        self.epochs = epochs
        self.model.compile(optimizer=Adam(lr=.003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.history = self.model.fit(training_gen,\
                                      validation_data=validation_data, \
                                      validation_steps= len(validation_data), \
                                      epochs=epochs, \
                                      initial_epoch=init_epoch,\
                                      callbacks=[reduce_lr, tensorboard], \
#                                       class_weight=self.class_weight, \
                                      verbose=1)


In [10]:
name = "same_global_local"
model_path = 'experiments/supervised/' + name + ".h5"
model = DoodleSDQModel(name, global_input_size=global_size, local_input_size=local_size, output_size=242)
# model.model.summary()
model.train(training_gen, valid_gen, 50, init_epoch=-1)
model.model.save(model_path)

Epoch 0/50
25/25 [==============================] - 3s 127ms/step - loss: 4.1075 - accuracy: 0.0688 - val_loss: 3.7853 - val_accuracy: 0.0882 - lr: 0.0030
Epoch 1/50
25/25 [==============================] - 3s 117ms/step - loss: 3.7070 - accuracy: 0.1136 - val_loss: 3.4965 - val_accuracy: 0.1481 - lr: 0.0030
Epoch 2/50
25/25 [==============================] - 3s 116ms/step - loss: 3.2632 - accuracy: 0.1563 - val_loss: 3.0105 - val_accuracy: 0.1686 - lr: 0.0030
Epoch 3/50
25/25 [==============================] - 3s 119ms/step - loss: 2.9992 - accuracy: 0.1774 - val_loss: 2.8955 - val_accuracy: 0.2004 - lr: 0.0030
Epoch 4/50
25/25 [==============================] - 3s 121ms/step - loss: 2.8840 - accuracy: 0.1868 - val_loss: 2.8446 - val_accuracy: 0.1872 - lr: 0.0030
Epoch 5/50
25/25 [==============================] - 3s 119ms/step - loss: 2.8282 - accuracy: 0.1902 - val_loss: 2.7499 - val_accuracy: 0.2070 - lr: 0.0030
Epoch 6/50
25/25 [==============================] - 3s 115ms/step - lo